In [2]:
import docx
import pandas as pd

In [3]:
#get the docx file
doc = docx.Document('publicServices.docx')

#get the title
title = doc.paragraphs[0].text

#get the publicServices they are in the first list of the document in a numbered list (180 items)
publicServices = []
for i in range(2, 182):
    publicServices.append(doc.paragraphs[i].text)
#now we go to the 'Details' section  which is a numbered list of the services with 
#for each publicService there is it's name on the top then:
#-'Description' a section
#-'Services proposés' a section
#-'Processus de candidature' a section
#-'Critères d'éligibilité ' a section
#each section has the name in bold with a ':' in the and and the content in the next line;the content can be in multiple lines


In [19]:
def getNextBoldLineFromLineNumber(lineNumber):
    currentLine = lineNumber+1
    while True:
        isBold= False
        try:
            isBold = doc.paragraphs[currentLine].runs[0].bold
            if isBold:
                return currentLine
            else:
                currentLine += 1
        except Exception as e:
            return -1
def getNextBoldLineWithNumberFromLineNumber(lineNumber):
    #get the next line with a bold text and a list number
    currentLine = lineNumber
    while True:
        found = False
        try:
            found = doc.paragraphs[currentLine].runs[0].bold and doc.paragraphs[currentLine].text.split('.')[0].isdigit()
            if found:
                return currentLine
            else:
                currentLine += 1
        except Exception as e:
            return -1
  

In [34]:
              
def getServiceDetails(titleLineNumber):
    details = {}
    details['name'] = doc.paragraphs[titleLineNumber].text.split('.')[1]
    currentLine = titleLineNumber+1
    nextDetailsBeginningLine = getNextBoldLineWithNumberFromLineNumber(currentLine)
    paragraphs=[
        {
            "key":'description',
            "title":'Description',
            
        },
        {
            'key':'services',
            "title":'Services proposés',
            
        },
        {
            'key':'process',
            'title':'Processus de candidature',
            
        },
        {
            'key':'eligibility',
            'title':'Critères d\'éligibilité',
        }
    ]
    for par in paragraphs:
        details[par['key']] = ""
        endingLine = getNextBoldLineFromLineNumber(currentLine)
        #print(endingLine)
        for lineNumber in range(currentLine,endingLine):
            details[par['key']] += doc.paragraphs[lineNumber].text
        details[par['key']] = details[par['key']].replace(par['title']+" :","")
        currentLine = endingLine
    
    return details,nextDetailsBeginningLine

In [35]:

publicServicesDetails = []
beginningLine = 186
#next service details beginning with a bold paragph with a number
nextDetailsBeginningLine = 186
print("nb paragraphs",len(doc.paragraphs))
while (nextDetailsBeginningLine > 0):
    service,nextDetailsBeginningLine = getServiceDetails(nextDetailsBeginningLine)
    #print(nextDetailsBeginningLine)
    publicServicesDetails.append(service)



nb paragraphs 751


In [ ]:


#export public services to excel 
df = pd.DataFrame(publicServicesDetails)
df.to_excel('publicServicesDetails.xlsx',index=False)

#print(publicServicesDetails[0])
